In [ ]:
!pip install ksql

In [ ]:
from ksql import KSQLAPI
ksql_client = KSQLAPI('http://ksqldb-server:8088')

## Start Sending smoke data via API

In [ ]:
rows = [
        {"sensor": "S1", "smoke": False, "ts": "2021-10-23T06:00:00+0200"},
        {"sensor": "S1", "smoke": True, "ts": "2021-10-23T06:00:01+0200"},
        {"sensor": "S1", "smoke": True, "ts": "2021-10-23T06:00:02+0200"},
        {"sensor": "S1", "smoke": True, "ts": "2021-10-23T06:00:03+0200"},
        {"sensor": "S1", "smoke": True, "ts": "2021-10-23T06:00:04+0200"}
    ]

results = ksql_client.inserts_stream("Smoke_STREAM", rows)

## And now, live data!

### causing no fire

In [ ]:
from IPython.display import clear_output
from datetime import datetime
import pytz
import time

amsterdam = pytz.timezone('Europe/Amsterdam')

while True:
    ts = datetime.now().astimezone(amsterdam).strftime("%Y-%m-%dT%H:%M:%S%z")
    row = [{"sensor": "S1","smoke": False,"ts": ts}]
    results = ksql_client.inserts_stream("Smoke_STREAM", row)
    print(row)
    time.sleep(10)
    clear_output(wait=True)

### causing fire (if also the temperature is high)

In [ ]:
from IPython.display import clear_output
from datetime import datetime
import pytz
import time

amsterdam = pytz.timezone('Europe/Amsterdam')

while True:
    ts = datetime.now().astimezone(amsterdam).strftime("%Y-%m-%dT%H:%M:%S%z")
    row = [{"sensor": "S1","smoke": True,"ts": ts}]
    results = ksql_client.inserts_stream("Smoke_STREAM", row)
    print(row)
    time.sleep(10)
    clear_output(wait=True)